In [ ]:
# This notebook computes the best fit and the number of events
# in the best fit for all possible test statistics.

import os
homedir = os.getcwd()[:-len('Notebooks')]
import sys
sys.path.append("..") # Needed to import files
import time

import MyUnits as muns
import MyFunctions as mfuns
import PlottingVariables as plvars
cmp = 1./2.54 # centimeters to inches for plotting
import Parameters as pars

import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from scipy.optimize import minimize, root
from scipy.stats import chi2, poisson

import Experiments.ANITA as AN
import Calculators.Probabilities as probs
import Experiments.KM3NeT as K3
import Experiments.IceCube as IC
import JointFits.ANITApK3pIC as AI3

plvars.add_style(homedir+'PlotData/style.mplstyle')
plotdirK = homedir + 'Figures/KM3NeT/'
datadirK = homedir + "Data/KM3NeT/"
datadirI = homedir + "Data/IceCube/"
datadirA = homedir + "Data/ANITA/"


In [ ]:
# INTERPOLATORS WITHOUT ABSORPTION
# Comment (or don't run this cell) if you want T to be absorbed.
########################################
# pars.is_T_absorbed = True
# K3.interp_aeff_mu = mfuns.create_interpolator_from_datafile(datadirK+"EffectiveArea_NoAbsorption.dat",2)
# K3.interp_aeff_NT = mfuns.create_interpolator_from_datafile(datadirK+"EffectiveArea_NoAbsorption.dat",(3,4))
# K3.interp_avgd_aeff = mfuns.create_interpolator_from_datafile(datadirK+"AveragedArea_NoAbsorption.dat",2)

# IC.interp_aeff_mu = mfuns.create_interpolator_from_datafile(datadirI+"EffectiveAreas_NoAbsorption.dat",2)
# IC.interp_aeff_NT = mfuns.create_interpolator_from_datafile(datadirI+"EffectiveAreas_NoAbsorption.dat",(3,4))

# anita_file = datadirA + "EffectiveAreas_NoAbsorption.dat"
# AN.interp_aeff_tot  = mfuns.create_interpolator_from_datafile(anita_file,2)
# AN.interp_avg1_aeff = mfuns.create_interpolator_from_datafile(anita_file,3)
# AN.interp_avg2_aeff = mfuns.create_interpolator_from_datafile(anita_file,4)
# AN.interp_avg3_aeff = mfuns.create_interpolator_from_datafile(anita_file,5)
# AN.interp_avg4_aeff = mfuns.create_interpolator_from_datafile(anita_file,6)
# AN.interps_avgs = [AN.interp_avg1_aeff,AN.interp_avg2_aeff,AN.interp_avg3_aeff,AN.interp_avg4_aeff]

# WE DON'T COUNT CASCADES
# If this variable is False, then we take into account that KM3NeT has not looked for cascade-like events
K3.count_cascades = False

# THE PRIMARY VERTEX IS NOT DETECTABLE
pars.N_detectable = True

In [3]:
##############################
# ANITA-only
##############################
def ts_anita(x):
    xs0, lt0 = np.exp(x)
    # xs, lt = AI3.vars_at_EeV(xs0,lt0)
    phi = AI3.BestFitDiffuseFluxANITA(xs0,lt0,0.0)
    return AI3.DiffuseTestStatisticANITA(xs0,lt0,0.0,phi)

x0 = np.array([1e-32,1e-3])
xs0bf, lt0bf = np.exp(minimize(ts_anita,np.log(x0)).x)
phibf = AI3.BestFitDiffuseFluxANITA(xs0bf,lt0bf,0.0)
xsbf, ltbf = AI3.vars_at_EeV(xs0bf,lt0bf)
eventsK3 = K3.DiffuseEvents(xsbf,ltbf,0.0,phibf,return_sum=False)
eventsIC = IC.DiffuseEvents(xsbf,ltbf,0.0,phibf,return_sum=False)
eventsAN = AN.DiffuseEvents(xsbf,ltbf,phibf)

print("ANITA:\txs = {:.2e} cm^2, lt = {:.2e} s, br = {:.3f},\n\tphi = {:.2e}/km^2/day = {:.2e}*GeV/cm^2/s/sr,\n\tN_K3 = ({:.2f},{:.2f},{:.2f}), N_IC = ({:.2f},{:.2f},{:.2f}), N_AN = {:.2f}".format(
        xsbf,ltbf,0.0,
        phibf*muns.km**2*muns.day,phibf*pars.EnergyDelta/muns.GeV/4/np.pi,
        eventsK3[0],eventsK3[1],eventsK3[2],
        eventsIC[0],eventsIC[1],eventsIC[2], 
        eventsAN))

ANITA:	xs = 2.19e-32 cm^2, lt = 4.87e-03 s, br = 0.000,
	phi = 4.59e+00/km^2/day = 2.11e-06*GeV/cm^2/s/sr,
	N_K3 = (0.00,0.15,0.00), N_IC = (0.00,9.62,0.00), N_AN = 4.05


In [4]:
##############################
# ANITA+ICECUBE
##############################
def ts_ANITApIC(x):
    xs0, lt0 = np.exp(x)
    # xs, lt = AI3.vars_at_EeV(xs0,lt0)
    phi = AI3.BestFitDiffuseFluxAI(xs0,lt0,0.0)
    return AI3.DiffuseTestStatisticAI(xs0,lt0,0.0,phi)

x0 = np.array([1e-32,1e-3])
xs0bf, lt0bf = np.exp(minimize(ts_ANITApIC,np.log(x0)).x)
phibf = AI3.BestFitDiffuseFluxAI(xs0bf,lt0bf,0.0)
xsbf, ltbf = AI3.vars_at_EeV(xs0bf,lt0bf)
eventsK3 = K3.DiffuseEvents(xsbf,ltbf,0.0,phibf,return_sum=False)
eventsIC = IC.DiffuseEvents(xsbf,ltbf,0.0,phibf,return_sum=False)
eventsAN = AN.DiffuseEvents(xsbf,ltbf,phibf)

print("IceCu:\txs = {:.2e} cm^2, lt = {:.2e} s, br = {:.3f},\n\tphi = {:.2e}/km^2/day = {:.2e}*GeV/cm^2/s/sr,\n\tN_K3 = ({:.2f},{:.2f},{:.2f}), N_IC = ({:.2f},{:.2f},{:.2f}), N_AN = {:.2f}".format(
        xsbf,ltbf,0.0,
        phibf*muns.km**2*muns.day,phibf*pars.EnergyDelta/muns.GeV/4/np.pi,
        eventsK3[0],eventsK3[1],eventsK3[2],
        eventsIC[0],eventsIC[1],eventsIC[2], 
        eventsAN))

IceCu:	xs = 1.96e-32 cm^2, lt = 2.38e-03 s, br = 0.000,
	phi = 3.55e+00/km^2/day = 1.63e-06*GeV/cm^2/s/sr,
	N_K3 = (0.00,0.11,0.00), N_IC = (0.00,6.76,0.00), N_AN = 4.69


In [3]:
##############################
# KM3NeT-only
##############################
def ts_KM3NeT(x):
    xs0, lt0 = np.exp(x)
    # xs, lt = AI3.vars_at_EeV(xs0,lt0)
    phi = AI3.BestFitDiffuseFluxKM3NeT(xs0,lt0,1.0)
    return AI3.DiffuseTestStatisticK3(xs0,lt0,1.0,phi)

x0 = np.array([1e-32,1e-3])
xs0bf, lt0bf = np.exp(minimize(ts_KM3NeT,np.log(x0)).x)
phibf = AI3.BestFitDiffuseFluxKM3NeT(xs0bf,lt0bf,1.0)
xsbf, ltbf = AI3.vars_at_EeV(xs0bf,lt0bf)
eventsK3 = K3.DiffuseEvents(xsbf,ltbf,1.0,phibf,return_sum=False)
eventsIC = IC.DiffuseEvents(xsbf,ltbf,1.0,phibf,return_sum=False)
eventsAN = 0.0#AN.DiffuseEvents(xsbf,ltbf,phibf)

print("KM3NeT:\txs = {:.2e} cm^2, lt = {:.2e} s, br = {:.3f},\n\tphi = {:.2e}/km^2/day = {:.2e}*GeV/cm^2/s/sr,\n\tN_K3 = ({:.2f},{:.2f},{:.2f}), N_IC = ({:.2f},{:.2f},{:.2f})".format(
        xsbf,ltbf,1.0,
        phibf*muns.km**2*muns.day,phibf*pars.EnergyDelta/muns.GeV/4/np.pi,
        eventsK3[0],eventsK3[1],eventsK3[2],
        eventsIC[0],eventsIC[1],eventsIC[2]),eventsAN)


KM3NeT:	xs = 2.23e-32 cm^2, lt = 1.03e-03 s, br = 1.000,
	phi = 2.06e+01/km^2/day = 9.50e-06*GeV/cm^2/s/sr,
	N_K3 = (0.91,0.64,0.09), N_IC = (26.48,59.65,4.89) 0.0


In [3]:
##############################
# KM3NeT+IceCube
##############################
def ts_K3pIC(x):
    xs0, lt0 = np.exp(x)
    # xs, lt = AI3.vars_at_EeV(xs0,lt0)
    br = 1.0
    phi = AI3.BestFitDiffuseFluxI3(xs0,lt0,br)
    return AI3.DiffuseTestStatisticI3(xs0,lt0,br,phi)

x0 = np.array([1e-32,1e-3])
xs0bf, lt0bf = np.exp(minimize(ts_K3pIC,np.log(x0)).x)
brbf = 1.0
phibf = AI3.BestFitDiffuseFluxI3(xs0bf,lt0bf,brbf)
xsbf, ltbf = AI3.vars_at_EeV(xs0bf,lt0bf)
eventsK3 = K3.DiffuseEvents(xsbf,ltbf,brbf,phibf,return_sum=False)
eventsIC = IC.DiffuseEvents(xsbf,ltbf,brbf,phibf,return_sum=False)
eventsAN = AN.DiffuseEvents(xsbf,ltbf,phibf)*(1-brbf)

print("IC+K3:\txs = {:.2e} cm^2, lt = {:.2e} s, br = {:.3f},\n\tphi = {:.2e}/km^2/day = {:.2e}*GeV/cm^2/s/sr,\n\tN_K3 = ({:.2f},{:.2f},{:.2f}), N_IC = ({:.2f},{:.2f},{:.2f}), N_AN = {:.2f}".format(
        xsbf,ltbf,brbf,
        phibf*muns.km**2*muns.day,phibf*pars.EnergyDelta/muns.GeV/4/np.pi,
        eventsK3[0],eventsK3[1],eventsK3[2],
        eventsIC[0],eventsIC[1],eventsIC[2],eventsAN))

print("ANITA-only flux: {:.5e}, K3-only flux: {:.5e}".format(AI3.BestFitDiffuseFluxANITA(xs0bf,lt0bf,0.0)*muns.km**2*muns.day,
                                                             AI3.BestFitDiffuseFluxKM3NeT(xs0bf,lt0bf,1.0)*muns.km**2*muns.day))

IC+K3:	xs = 5.89e-33 cm^2, lt = 1.04e-04 s, br = 1.000,
	phi = 2.49e+00/km^2/day = 1.15e-06*GeV/cm^2/s/sr,
	N_K3 = (0.17,0.02,0.01), N_IC = (4.52,2.01,0.64), N_AN = 0.00
ANITA-only flux: 2.94681e+01, K3-only flux: 2.16253e+01


In [4]:
##############################
# KM3NeT+ANITA
##############################
def ts_ANITApK3(x):
    xs0, lt0 = np.exp(x)
    # xs, lt = AI3.vars_at_EeV(xs0,lt0)
    br = AI3.BestFitBranchingRatioNoPhiA3(xs0,lt0)
    phi = AI3.BestFitDiffuseFluxA3(xs0,lt0,br)
    return AI3.DiffuseTestStatisticA3(xs0,lt0,br,phi)

x0 = np.array([1e-32,1e-3])
xs0bf, lt0bf = np.exp(minimize(ts_ANITApK3,np.log(x0)).x)
brbf = AI3.BestFitBranchingRatioNoPhiA3(xs0bf,lt0bf)
phibf = AI3.BestFitDiffuseFluxA3(xs0bf,lt0bf,brbf)
xsbf, ltbf = AI3.vars_at_EeV(xs0bf,lt0bf)
eventsK3 = K3.DiffuseEvents(xsbf,ltbf,brbf,phibf,return_sum=False)
eventsIC = IC.DiffuseEvents(xsbf,ltbf,brbf,phibf,return_sum=False)
eventsAN = AN.DiffuseEvents(xsbf,ltbf,phibf)*(1-brbf)

print("AN+K3:\txs = {:.2e} cm^2, lt = {:.2e} s, br = {:.3f},\n\tphi = {:.2e}/km^2/day = {:.2e}*GeV/cm^2/s/sr,\n\tN_K3 = ({:.2f},{:.2f},{:.2f}), N_IC = ({:.2f},{:.2f},{:.2f}), N_AN = {:.2f}".format(
        xsbf,ltbf,brbf,
        phibf*muns.km**2*muns.day,phibf*pars.EnergyDelta/muns.GeV/4/np.pi,
        eventsK3[0],eventsK3[1],eventsK3[2],
        eventsIC[0],eventsIC[1],eventsIC[2],eventsAN))

print("ANITA-only flux: {:.5e}, K3-only flux: {:.5e}".format(AI3.BestFitDiffuseFluxANITA(xs0bf,lt0bf,0.0)*muns.km**2*muns.day,
                                                             AI3.BestFitDiffuseFluxKM3NeT(xs0bf,lt0bf,1.0)*muns.km**2*muns.day))

AN+K3:	xs = 2.82e-32 cm^2, lt = 7.16e-04 s, br = 0.902,
	phi = 2.06e+01/km^2/day = 9.47e-06*GeV/cm^2/s/sr,
	N_K3 = (1.16,0.81,0.11), N_IC = (32.11,75.17,6.14), N_AN = 2.33
ANITA-only flux: 2.01946e+00, K3-only flux: 1.85427e+01


In [5]:
##############################
# KM3NeT+ANITA+IC
##############################
def ts_ANITApK3pIC(x):
    xs0, lt0 = np.exp(x)
    # xs, lt = AI3.vars_at_EeV(xs0,lt0)
    br = AI3.BestFitBranchingRatioNoPhiAI3(xs0,lt0)
    phi = AI3.BestFitDiffuseFluxAI3(xs0,lt0,br)
    return AI3.DiffuseTestStatisticAI3(xs0,lt0,br,phi)

x0 = np.array([1e-32,1e-3])
xs0bf, lt0bf = np.exp(minimize(ts_ANITApK3pIC,np.log(x0)).x)
brbf = AI3.BestFitBranchingRatioNoPhiAI3(xs0bf,lt0bf)
phibf = AI3.BestFitDiffuseFluxAI3(xs0bf,lt0bf,brbf)
xsbf, ltbf = AI3.vars_at_EeV(xs0bf,lt0bf)
eventsK3 = K3.DiffuseEvents(xsbf,ltbf,brbf,phibf,return_sum=False)
eventsIC = IC.DiffuseEvents(xsbf,ltbf,brbf,phibf,return_sum=False)
eventsAN = AN.DiffuseEvents(xsbf,ltbf,phibf)*(1-brbf)

print("ANK3IC:\txs = {:.2e} cm^2, lt = {:.2e} s, br = {:.3f},\n\tphi = {:.2e}/km^2/day = {:.2e}*GeV/cm^2/s/sr,\n\tN_K3 = ({:.2f},{:.2f},{:.2f}), N_IC = ({:.2f},{:.2f},{:.2f}), N_AN = {:.2f}".format(
        xsbf,ltbf,brbf,
        phibf*muns.km**2*muns.day,phibf*pars.EnergyDelta/muns.GeV/4/np.pi,
        eventsK3[0],eventsK3[1],eventsK3[2],
        eventsIC[0],eventsIC[1],eventsIC[2],eventsAN))


print("ANITA-only flux: {:.5e}, K3-only flux: {:.5e}".format(AI3.BestFitDiffuseFluxANITA(xs0bf,lt0bf,0.0)*muns.km**2*muns.day,
                                                             AI3.BestFitDiffuseFluxKM3NeT(xs0bf,lt0bf,1.0)*muns.km**2*muns.day))

ANK3IC:	xs = 5.93e-33 cm^2, lt = 1.01e-03 s, br = 0.437,
	phi = 8.63e+00/km^2/day = 3.97e-06*GeV/cm^2/s/sr,
	N_K3 = (0.12,0.09,0.01), N_IC = (4.67,7.83,0.59), N_AN = 5.94
ANITA-only flux: 2.68858e+00, K3-only flux: 3.71492e+01
